In [1]:
# 공공 데이터 포탈에서 다운받은 '소상공인시장진흥공단_상가(상권)정보_대구_202312.csv'를 사용

# 1. 필요한 정보만 데이터 프레임으로 변경
# 2. 남구, 달서구, 상권업종중분류명 : 이용-미용 지도에 표시

In [2]:
import pandas as pd
import folium
import json

In [3]:
df = pd.read_csv('./input/소상공인시장진흥공단_상가(상권)정보_대구_202312.csv')
df.head()

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,MA0101202210A0090318,아보카,NaN,I2,음식,I203,일식,I20301,일식 회/초밥,I56122,...,2729011600110100000000001,NaN,대구광역시 달서구 수밭동길 34,704802,42835,NaN,1,NaN,128.549749,35.795445
1,MA010120220805430943,모텔탑,NaN,I1,숙박,I101,일반 숙박,I10102,여관/모텔,I55102,...,2714014200100370005004100,NaN,대구광역시 동구 파계로138길 40,701500,41001,NaN,NaN,NaN,128.639751,35.992577
2,MA010120220805432876,킴스헤어,NaN,S2,수리·개인,S207,이용·미용,S20701,미용실,S96112,...,2723011400106790000009408,대구동서변그린빌,대구광역시 북구 동변로 55,702733,41414,NaN,1,NaN,128.602432,35.916799
3,MA010120220805435697,카페사람이야기,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,2714012300110990000000001,NaN,대구광역시 동구 이노밸리로46길 4-9,701290,41067,NaN,1,NaN,128.721839,35.877366
4,MA010120220805435246,다즐링,NaN,I2,음식,I212,비알코올,I21201,카페,I56221,...,2714010200103510034044815,NaN,대구광역시 동구 화랑로 23,701829,41252,NaN,1,NaN,128.631047,35.868657


In [22]:
# 필요한 정보만 데이터 프레임으로 변경 (시군구명, 상권업종중분류명, 위도, 경도)
df1 = df[['상권업종중분류명', '시군구명', '위도', '경도', '상호명']]
df1.head()

,상권업종중분류명,시군구명,위도,경도,상호명
0,일식,달서구,35.795445,128.549749,아보카
1,일반 숙박,동구,35.992577,128.639751,모텔탑
2,이용·미용,북구,35.916799,128.602432,킴스헤어
3,비알코올,동구,35.877366,128.721839,카페사람이야기
4,비알코올,동구,35.868657,128.631047,다즐링


In [23]:
# 남구, 달서구 구분
df2 = df1[(df1['시군구명'] == '달서구') | (df['시군구명'] == '남구')]
df2.head()

,상권업종중분류명,시군구명,위도,경도,상호명
0,일식,달서구,35.795445,128.549749,아보카
5,의약·화장품 소매,달서구,35.851705,128.573055,이오스타
6,기술 서비스,남구,35.854293,128.598704,건축사사무소토마
8,한식,달서구,35.823586,128.527333,공원생막창&뒷고기
11,일반 숙박,남구,35.846831,128.580293,야크모텔


In [24]:
df3 = df2[df['상권업종중분류명'] == '이용·미용']
df3.head()

/var/folders/ch/9dhkv5f92c353r15wr2y5d800000gn/T/ipykernel_6123/720146666.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df3 = df2[df['상권업종중분류명'] == '이용·미용']


,상권업종중분류명,시군구명,위도,경도,상호명
28,이용·미용,달서구,35.819959,128.524078,티나나
35,이용·미용,달서구,35.825240,128.526226,라플레르헤어
37,이용·미용,남구,35.837112,128.565578,렌부띠끄
79,이용·미용,달서구,35.848738,128.536400,엠코이
83,이용·미용,남구,35.849612,128.571556,보리떼모노


In [27]:
df3.dtypes

상권업종중분류명     object
시군구명         object
위도          float64
경도          float64
상호명          object
dtype: object

In [29]:
df3.columns = df3.columns.map(str)

In [28]:
df.dtypes

상가업소번호        object
상호명           object
지점명           object
상권업종대분류코드     object
상권업종대분류명      object
상권업종중분류코드     object
상권업종중분류명      object
상권업종소분류코드     object
상권업종소분류명      object
표준산업분류코드      object
표준산업분류명       object
시도코드           int64
시도명           object
시군구코드          int64
시군구명          object
행정동코드          int64
행정동명          object
법정동코드          int64
법정동명          object
지번코드           int64
대지구분코드         int64
대지구분명         object
지번본번지        float64
지번부번지        float64
지번주소          object
도로명코드        float64
도로명           object
건물본번지        float64
건물부번지        float64
건물관리번호        object
건물명           object
도로명주소         object
구우편번호          int64
신우편번호          int64
동정보          float64
층정보           object
호정보          float64
경도           float64
위도           float64
dtype: object

In [35]:
# 대구광역시 중구 : 35.8683° N 위도, 128.5988° E 경도
daegu_map = folium.Map(location=[35.8683, 128.5988], zoom_start=12, tiles='Cartodb Positron')
daegu_map

In [36]:
for name, lat, lng in zip(df3['상호명'], df3['위도'], df3['경도']):
    folium.Marker([lat, lng], popup=name).add_to(daegu_map)
daegu_map

In [37]:
daegu_map.save('./output/daegu_map.html')